# <center> Pretrained Models - TranferLearning </center>



In [1]:
import cv2 
import os 
import glob 
import matplotlib.pyplot as plt
import numpy             as np
import tensorflow        as tf
import sklearn           as sk

from tensorflow.keras.models                import Sequential, Model
from tensorflow.keras.layers                import Dense, Input, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.layers                import BatchNormalization, Dropout, Reshape, Activation, ReLU
from tensorflow.keras.layers                import Conv2DTranspose, UpSampling2D, MaxPooling2D
from tensorflow.keras.preprocessing.image   import ImageDataGenerator
from tensorflow.keras.utils                 import to_categorical
from tensorflow.keras.optimizers            import SGD

from skimage.transform       import resize
from sklearn.model_selection import train_test_split



In [6]:

def loadImagesDirectory(directory):
    files =[x[0] for x in os.walk(directory)]  # uses carefully, is long dataset
    data = []
    labels = []
    label = -1
    print(files[1:])
    for f1 in files[1:]:
        label += 1
        
        data_path = os.path.join(f1, '*g') 
        files_ = glob.glob(data_path) 
        
        for image in files_:
            img = cv2.imread(image)
            if img is None:
                continue
            res = cv2.resize(img, dsize=(150, 150), interpolation=cv2.INTER_CUBIC)
            data.append(res)
            
        labels += ([label] * (len(data) - len(labels)))
    
    return np.array(data), np.array(labels)

In [7]:
x_train, y_train = loadImagesDirectory("./PetImages/train")

x_test, y_test = loadImagesDirectory("./PetImages/test") 


['./PetImages/train\\cat', './PetImages/train\\dog']
['./PetImages/test\\cat', './PetImages/test\\dog']


In [8]:
batch_size = 32

Y_train = tf.keras.utils.to_categorical(y_train)
Y_test  = tf.keras.utils.to_categorical(y_test)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


train_datagen.fit(x_train)
test_datagen.fit(x_train)

In [5]:
model = Sequential()

model.add(Conv2D(16, kernel_size=(2,2), strides=1))
model.add(BatchNormalization())

model.add(ReLU())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))


model.add(Conv2D(32,kernel_size=(2,2), strides=1))
model.add(BatchNormalization())

model.add(ReLU())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))


model.add(Conv2D(64,kernel_size=(2,2), strides=1))
model.add(BatchNormalization())

model.add(ReLU())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))


model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(2, activation='softmax'))

In [6]:

model.compile(optimizer=SGD(0.05), loss='binary_crossentropy',  metrics=['accuracy'])

In [10]:
logdir="logs/Convolutional-dog-cat-profiler"


# tf.profiler.experimental.start(logdir=logdir)

metricsCallback = tf.keras.callbacks.TensorBoard(log_dir="logs/Convolutional-dog-cat-metrics")

model.fit(train_datagen.flow(x_train, Y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32,
          epochs=20,
          batch_size=batch_size,
          callbacks=[metricsCallback],
          validation_data=test_datagen.flow(x_test, Y_test, batch_size=32))

# tf.profiler.experimental.stop()

Epoch 1/20
554/553 [==============================] - 77s 139ms/step - loss: 0.3006 - accuracy: 0.8692 - val_loss: 0.3737 - val_accuracy: 0.8349
Epoch 2/20
554/553 [==============================] - 77s 138ms/step - loss: 0.2989 - accuracy: 0.8706 - val_loss: 0.3852 - val_accuracy: 0.8394
Epoch 3/20
554/553 [==============================] - 76s 138ms/step - loss: 0.2857 - accuracy: 0.8785 - val_loss: 0.3797 - val_accuracy: 0.8429
Epoch 4/20
554/553 [==============================] - 77s 138ms/step - loss: 0.2836 - accuracy: 0.8785 - val_loss: 0.3847 - val_accuracy: 0.8314
Epoch 5/20
554/553 [==============================] - 79s 142ms/step - loss: 0.2792 - accuracy: 0.8821 - val_loss: 0.3645 - val_accuracy: 0.8416
Epoch 6/20
554/553 [==============================] - 78s 140ms/step - loss: 0.2702 - accuracy: 0.8828 - val_loss: 0.3989 - val_accuracy: 0.8299
Epoch 7/20
554/553 [==============================] - 76s 138ms/step - loss: 0.2658 - accuracy: 0.8885 - val_loss: 0.3774 - val_ac

In [9]:
model.save("./models-save/DogCat-Convolutional-86-83")

INFO:tensorflow:Assets written to: ./models-save/DogCat-Convolutional-86-83\assets


In [ ]:
## tranfer learning

In [102]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

68608000/68606236 [==============================] - 33s 0us/step


In [124]:


model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False)


In [125]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory("./cats_and_dogs_filtered/train",
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(160, 160))

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory("./cats_and_dogs_filtered/validation",
                                                  shuffle=True,
                                                  batch_size=32,
                                                  image_size=(160, 160))

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [126]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = model(image_batch)
print(feature_batch.shape)

model.trainable = False

(32, 5, 5, 512)


In [127]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 26
Number of test batches: 6


In [128]:
# Use buffered prefetching to load images from disk without having I/O become blocking

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [129]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 512)


In [130]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 1)


In [131]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [132]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [133]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [134]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         validation_data=validation_dataset)

Epoch 1/20
63/63 [==============================] - 6s 102ms/step - loss: 2.8953 - accuracy: 0.5810 - val_loss: 1.8851 - val_accuracy: 0.6485
Epoch 2/20
63/63 [==============================] - 6s 99ms/step - loss: 2.4036 - accuracy: 0.6230 - val_loss: 1.4234 - val_accuracy: 0.7116
Epoch 3/20
63/63 [==============================] - 6s 99ms/step - loss: 2.0133 - accuracy: 0.6525 - val_loss: 1.1480 - val_accuracy: 0.7525
Epoch 4/20
63/63 [==============================] - 6s 99ms/step - loss: 1.8778 - accuracy: 0.6855 - val_loss: 0.9021 - val_accuracy: 0.7958
Epoch 5/20
63/63 [==============================] - 6s 99ms/step - loss: 1.7258 - accuracy: 0.6860 - val_loss: 0.8663 - val_accuracy: 0.8106
Epoch 6/20
63/63 [==============================] - 6s 101ms/step - loss: 1.4621 - accuracy: 0.7305 - val_loss: 0.7500 - val_accuracy: 0.8391
Epoch 7/20
63/63 [==============================] - 6s 99ms/step - loss: 1.4943 - accuracy: 0.7240 - val_loss: 0.6728 - val_accuracy: 0.8441
Epoch 8/20


In [135]:
model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[:fine_tune_at]:
  layer.trainable =  False



Number of layers in the base model:  8


In [136]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:


!tensorboard --logdir "D:\La cosa de Trabajo\WorkSpaces\Jupyter\Notebook  Colab\logs\MINIS-convolutional-metrics"
--port 909



In [137]:
fine_tune_epochs = 10
total_epochs =  10 + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         validation_data=validation_dataset)

Epoch 1/20
63/63 [==============================] - 6s 101ms/step - loss: 0.6919 - accuracy: 0.8460 - val_loss: 0.3095 - val_accuracy: 0.9307
Epoch 2/20
63/63 [==============================] - 6s 99ms/step - loss: 0.6708 - accuracy: 0.8495 - val_loss: 0.3331 - val_accuracy: 0.9257
Epoch 3/20
63/63 [==============================] - 6s 98ms/step - loss: 0.6693 - accuracy: 0.8400 - val_loss: 0.3176 - val_accuracy: 0.9282
Epoch 4/20
63/63 [==============================] - 6s 91ms/step - loss: 0.7338 - accuracy: 0.8450 - val_loss: 0.3142 - val_accuracy: 0.9270
Epoch 5/20
63/63 [==============================] - 6s 89ms/step - loss: 0.6117 - accuracy: 0.8510 - val_loss: 0.3307 - val_accuracy: 0.9220
Epoch 6/20
63/63 [==============================] - 6s 92ms/step - loss: 0.6590 - accuracy: 0.8580 - val_loss: 0.2992 - val_accuracy: 0.9270
Epoch 7/20
63/63 [==============================] - 6s 89ms/step - loss: 0.6757 - accuracy: 0.8510 - val_loss: 0.2982 - val_accuracy: 0.9282
Epoch 8/20
6